### Fine-Tuning LLaMA-3.2 3B Instruct

This code will fine-tune the `LLaMA-3.2-3B-Instruct` model on the CounselChat dataset

In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import torch
import pickle
from unsloth import FastLanguageModel, is_bfloat16_supported, train_on_responses_only
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict
from trl import SFTTrainer

Some links to be used for fine-tuning
1. https://www.kdnuggets.com/fine-tuning-llama-using-unsloth
2. https://www.analyticsvidhya.com/blog/2024/12/fine-tuning-llama-3-2-3b-for-rag/
3. https://www.linkedin.com/pulse/step-guide-use-fine-tune-llama-32-dr-oualid-soula-xmnff/
4. https://medium.com/@alexandros_chariton/how-to-fine-tune-llama-3-2-instruct-on-your-own-data-a-detailed-guide-e5f522f397d7 (check the data format since it uses generation_token at end of input data since that might not be needed)
5. https://drlee.io/step-by-step-guide-fine-tuning-metas-llama-3-2-1b-model-f1262eda36c8
6. https://huggingface.co/blog/ImranzamanML/fine-tuning-1b-llama-32-a-comprehensive-article
7. https://blog.futuresmart.ai/fine-tune-llama-32-vision-language-model-on-custom-datasets
8. https://github.com/meta-llama/llama-cookbook/blob/main/getting-started/finetuning/multigpu_finetuning.md



### Reading the Counsel Chat Dataset

In [ ]:
with open('processed_data/counselchat_top_votes_train.pkl', 'rb') as file:
    dataset_top_votes_train = pickle.load(file)

dataset_top_votes_train.head()

In [ ]:
with open('processed_data/counselchat_top_votes_test.pkl', 'rb') as file:
    dataset_top_votes_test = pickle.load(file)

dataset_top_votes_test.head()

Creating the HuggingFace dataset using Pandas Dataframe

In [ ]:
dataset_train = Dataset.from_pandas(dataset_top_votes_train)
dataset_test = Dataset.from_pandas(dataset_top_votes_test)

In [ ]:
dataset = DatasetDict()
dataset['train'] = dataset_train
dataset['test'] = dataset_test

In [ ]:
dataset

### Fine-Tuning Code

#### Loading the model and tokenizer

In [ ]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

#### Setting up the PEFT settings for the model

https://huggingface.co/blog/damjan-k/rslora\
https://medium.com/@fartypantsham/what-rank-r-and-alpha-to-use-in-lora-in-llm-1b4f025fd133

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0.1,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 42,
    use_rslora = True,
    loftq_config = None,
)

#### Forming the chat template

In [ ]:
# Define a function to apply the chat template
def format_chat_template(example):
        
    messages = [
        {"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": example['answerText']}
    ]
    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False)

    return {"text": prompt}

In [ ]:
dataset_formatted = dataset.map(format_chat_template)

In [ ]:
print(dataset_formatted['train']['text'][0])

#### Initializing the TRL SFTTrainer and related Arguments

In [ ]:
peft_model_path = "./llama32-sft-peft-counselchat"

training_args = TrainingArguments(
        output_dir=peft_model_path,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        # gradient_accumulation_steps=4,
        eval_strategy="steps",
        eval_steps=0.1,
        logging_strategy="steps",
        logging_steps=50,
        save_strategy="steps",
        save_steps=0.1,
        warmup_steps = 5,
        num_train_epochs = 3,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        seed = 42,
        report_to = "none",
    )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset_formatted["train"],
    eval_dataset=dataset_formatted["test"],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer), #only use when using train_on_responses_only()
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = training_args)

In [ ]:
trainer.train_dataset

In [ ]:
print(tokenizer.decode(trainer.train_dataset['input_ids'][0]))

#### Only Focus on the `Response Part` for the generation

In [ ]:
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

In [ ]:
trainer.train_dataset

In [ ]:
# The labels are created which only contain response. Left Padding is implemented and all the padding tokens are given a score of -100 to avoid loss calculation for pad_tokens
trainer.train_dataset['labels'][0]

#### Train the model

In [ ]:
trainer_stats = trainer.train()

#### Saving the model and tokenizer

Merge the LoRA into the base model and then save 16-bit version

In [ ]:
merged_model_path = "./llama32-sft-merged-counselchat"
# model.save_pretrained_merged(merged_model_path, tokenizer, save_method = "merged_16bit")

Just Save LoRA Adapters

In [ ]:
peft_model_path = "./llama32-sft-peft-counselchat"
# model.save_pretrained_merged(peft_model_path, tokenizer, save_method = "lora",) #WJust Save LoRA Adapters

# Or run the two below statements
model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

### Inference

In [ ]:
merged_model_path = "./llama32-sft-merged-counselchat"
peft_model_path = "./llama32-sft-peft-counselchat"
sft_model = merged_model_path

In [ ]:
max_seq_length = 2048 
dtype = None # None for auto-detection.
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = sft_model,
    max_seq_length = max_seq_length,
    load_in_4bit=load_in_4bit,
    dtype=dtype,
    device_map="auto"
)

In [ ]:
idx=0

print(dataset['train']['question'][idx])

FastLanguageModel.for_inference(model)

messages = [{"role": "system", "content": "You are an expert mental health professional trained to counsel and guide patients suffering from ill mental-health"},
    {"role": "user", "content": dataset['train']['question'][idx]}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=2048, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

print('---------------------------------------------------')